In [1]:
from torch.utils.data import DataLoader
from dataset import AudioDataset
from config import TRAIN_DATASET, N_MFCC, NUM_EPOCHS, BATCH_SICE

In [2]:
train_audio_dataset = AudioDataset(TRAIN_DATASET)
train_loader = DataLoader(train_audio_dataset, batch_size=BATCH_SICE, shuffle=True)

In [ ]:
train_loader = DataLoader(train_audio_dataset, batch_size=15, shuffle=True)

In [3]:
import torch.nn as nn

class SpeakerRecognitionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SpeakerRecognitionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

model = SpeakerRecognitionModel(input_size=N_MFCC, hidden_size=128, num_layers=2, num_classes=len(set(train_audio_dataset.dataset['speaker'])))
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch .optim.Adam(model.parameters(), lr = 0.001)

CUDA is available. Using GPU.


In [5]:
for epoch in range(NUM_EPOCHS):
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 23.77 GiB. GPU 0 has a total capacity of 10.00 GiB of which 7.40 GiB is free. Of the allocated memory 1.43 GiB is allocated by PyTorch, and 1.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)